## Install prerequisites

In [1]:
from importlib.metadata import version
version('torch')

'2.4.0'

In [ ]:
pip install --upgrade transformers

https://docs.vllm.ai/en/latest/

In [2]:
version('transformers')

'4.44.2'

In [ ]:
pip install --upgrade vllm

In [3]:
version('vllm')

'0.6.1.post2'

https://sglang.readthedocs.io/en/latest/

In [ ]:
pip install --upgrade "sglang[all]"

In [ ]:
pip install flashinfer -i https://flashinfer.ai/whl/cu124/torch2.4/

In [4]:
version('sglang')

'0.3.1.post2'

In [5]:
version('triton')

'3.0.0'

In [6]:
version('flashinfer')

'0.1.6+cu124torch2.4'

https://github.com/ollama/ollama-python

https://github.com/ollama/ollama/tree/main/docs

In [ ]:
pip install --upgrade ollama

In [9]:
version('ollama')

'0.3.3'

### Format prompts

In [1]:
test_messages = [
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages du Crédit Mutuel ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages du Crédit Agricole ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la Société Générale ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la BNP ?"}
]
]

In [2]:
test_models = {                                                                    # OpenLLM leaderboard score
    "llama-3.1" : "meta-llama/Meta-Llama-3.1-8B-Instruct",                         # 100.0 %
    "llama-3.1:w8a16" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16",  # 99.8 %    
    "llama-3.1:fp8" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-FP8-dynamic",        # 99.7 %
    "llama-3.1:w8a8" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a8",    # 99.4 %
    "llama-3.1:w4a16" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16"   # 97.1 %
}

In [3]:
from transformers import AutoTokenizer

def format_prompt(messages, model):
    tokenizer = AutoTokenizer.from_pretrained(model)
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [4]:
format_prompt(test_messages, test_models["llama-3.1"])

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nTu es un assistant utile et professionnel qui répond toujours en français.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuels sont les avantages du Crédit Mutuel ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nTu es un assistant utile et professionnel qui répond toujours en français.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuels sont les avantages du Crédit Agricole ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nTu es un assistant utile et professionnel qui répond toujours en français.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuels sont 

### vLLM

In [14]:
# Authenticate VLLM with Huggingface Hub
import os

with open("/workspace/hftoken", 'r') as file:
    myhftoken = file.read().strip()

os.environ["HF_TOKEN"]=myhftoken

In [15]:
import time
from vllm import LLM, SamplingParams

def vllm_load(model):    
    llm = LLM(model, gpu_memory_utilization=0.99, max_model_len=8192)
    llm._model = model
    return llm

def vllm_generate(messages, llm):    
    print(f"vLLM performance test:")
    
    prompts = format_prompt(messages, llm._model)
    sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=512)
    # warmup
    outputs = llm.generate(prompts[0], sampling_params)
    print(f"Generated text: {outputs[0].outputs[0].text!r}")
    
    for batch_size in range(1, len(messages) + 1):
        start_time = time.time()  # Record the start time
        outputs = llm.generate(prompts[0:batch_size], sampling_params)
        end_time = time.time()  # Record the end time
            
        # Print the outputs.
        tokenscount = 0
        for output in outputs:
            generated_text = output.outputs[0].text
            tokenscount = tokenscount + len(output.outputs[0].token_ids)

        tokens_per_sec = tokenscount/(end_time-start_time)
        print(f"- batch size {batch_size}: {tokens_per_sec:.2f} tokens/sec ({batch_size} x {tokens_per_sec/batch_size:.2f})")

In [16]:
llm = vllm_load(test_models["llama-3.1"])

INFO 09-21 15:15:37 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llama/Meta-Llama-3.1-8B-Instruct, use_v2_block_manager=False, num_scheduler_steps=1, en

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 09-21 15:15:41 model_runner.py:1008] Loading model weights took 14.9888 GB
INFO 09-21 15:15:42 gpu_executor.py:122] # GPU blocks: 3610, # CPU blocks: 2048
INFO 09-21 15:15:42 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-21 15:15:42 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-21 15:15:52 model_runner.py:1430] Graph capturing finished in 10 secs.


In [17]:
vllm_generate(test_messages*8, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:09<00:00,  9.58s/it, est. speed input: 6.58 toks/s, output: 53.47 toks/s]


Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses membres et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés sur les épargnes** : Le Crédit Mutuel propose des taux d'intérêt plus élevés sur les comptes d'épargne que de nombreux autres établissements bancaires.\n2. **Taux d'emprunt compétitifs** : Les prêts personnels, les prêts immobiliers et les prêts pour la mobilité sont proposés avec des taux d'intérêt attractifs.\n3. **Services personnalisés** : Le Crédit Mutuel offre des services personnalisés et adaptés aux besoins de ses membres et clients, grâce à une approche relationnelle et à une connaissance approfondie de leurs situations financières.\n4. **Sécurité et confidentialité** : Le Crédit Mutuel s'engage à protéger les données personnelles et financières de ses membres et clients, conformément aux règles de protection des données.\n5. **Participation aux décisions** : En tant que membre 

Processed prompts: 100%|█████████████| 1/1 [00:09<00:00,  9.30s/it, est. speed input: 6.78 toks/s, output: 55.07 toks/s]


- batch size 1: 55.06 tokens/sec (1 x 55.06)


...

- batch size 30: 1242.63 tokens/sec (30 x 41.42)


Processed prompts: 100%|███████| 31/31 [00:12<00:00,  2.53it/s, est. speed input: 159.80 toks/s, output: 1282.74 toks/s]


- batch size 31: 1282.23 tokens/sec (31 x 41.36)


Processed prompts: 100%|███████| 32/32 [00:12<00:00,  2.56it/s, est. speed input: 161.42 toks/s, output: 1308.09 toks/s]

- batch size 32: 1307.55 tokens/sec (32 x 40.86)


### SGLang

In [5]:
import json, time
import sglang

def sglang_load(model):
    runtime = sglang.Runtime(model_path=model)
    runtime._model = model
    return runtime

def sglang_generate(messages, runtime):
    print(f"SGLang performance test:")
    
    prompts = format_prompt(messages, runtime._model)
    sampling_params = { "temperature":0.7, "top_p":0.8, "repetition_penalty":1.05, "max_new_tokens":512 }
    # warmup
    output = json.loads(runtime.generate(prompt=prompts[0], sampling_params=sampling_params))
    print(f"Generated text: {output['text']!r}")
    
    for batch_size in range(1, len(messages) + 1):
        start_time = time.time()  # Record the start time
        outputs = json.loads(runtime.generate(prompt=prompts[0:batch_size], sampling_params=sampling_params))
        end_time = time.time()  # Record the end time
            
        # Print the outputs.
        tokenscount = 0
        for output in outputs:
            generated_text = output["text"]
            tokenscount = tokenscount + output["meta_info"]["completion_tokens"]

        tokens_per_sec = tokenscount/(end_time-start_time)
        print(f"- batch size {batch_size}: {tokens_per_sec:.2f} tokens/sec ({batch_size} x {tokens_per_sec/batch_size:.2f})")

In [ ]:
runtime = sglang_load(test_models["llama-3.1"])

In [7]:
sglang_generate(test_messages*8, runtime)

SGLang performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre divers avantages à ses membres et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés sur les dépôts** : Le Crédit Mutuel propose des taux d'intérêt plus élevés que les banques traditionnelles pour les dépôts, ce qui permet aux clients de gagner plus d'argent sur leurs économies.\n\n2. **Taux d'emprunt compétitifs** : Les emprunts du Crédit Mutuel sont souvent moins chers que ceux proposés par les banques traditionnelles, ce qui peut aider les clients à se procurer des prêts à des conditions avantageuses.\n\n3. **Services personnalisés** : En tant que banque coopérative, le Crédit Mutuel prend en compte les besoins spécifiques de ses clients et offre des services personnalisés pour répondre à leurs attentes.\n\n4. **Transparence et sécurité** : Le Crédit Mutuel est connu pour sa transparence financière et sa sécurité, ce qui rassure les clients qui s

### Ollama

https://github.com/ollama/ollama/blob/main/docs/linux.md#manual-install

In [11]:
!mkdir ollama && curl -L https://ollama.com/download/ollama-linux-amd64.tgz -o ollama-linux-amd64.tgz && tar -C ./ollama -xzf ollama-linux-amd64.tgz && rm ollama-linux-amd64.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   117  100   117    0     0    566      0 --:--:-- --:--:-- --:--:--   567
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1583M  100 1583M    0     0  17.1M      0  0:01:32  0:01:32 --:--:-- 16.9M


```bash
./ollama/bin/ollama serve &
```

https://github.com/ollama/ollama-python

In [16]:
import ollama

ollama.list()

{'models': []}

https://ollama.com/library/llama3.1:8b-instruct-fp16

In [37]:
ollama_test_models = {
    "llama-3.1" : "llama3.1:8b-instruct-fp16",
    "llama-3.1:int8" : "llama3.1:8b-instruct-q8_0",
    "llama-3.1:int4" : "llama3.1:8b-instruct-q4_0"
}

```bash
./ollama/bin/ollama pull llama3.1:8b-instruct-fp16
```

In [20]:
ollama.list()

{'models': [{'name': 'llama3.1:8b-instruct-fp16',
   'model': 'llama3.1:8b-instruct-fp16',
   'modified_at': '2024-09-21T22:54:30.926572546+02:00',
   'size': 16068910253,
   'digest': '4aacac4194543ff7f70dab3f2ebc169c132d5319bb36f7a7e99c4ff525ebcc09',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '8.0B',
    'quantization_level': 'F16'}}]}

In [27]:
print(ollama.show("llama3.1:8b-instruct-fp16")['template'])

{{- if or .System .Tools }}<|start_header_id|>system<|end_header_id|>
{{- if .System }}

{{ .System }}
{{- end }}
{{- if .Tools }}

Cutting Knowledge Date: December 2023

When you receive a tool call response, use the output to format an answer to the orginal user question.

You are a helpful assistant with tool calling capabilities.
{{- end }}<|eot_id|>
{{- end }}
{{- range $i, $_ := .Messages }}
{{- $last := eq (len (slice $.Messages $i)) 1 }}
{{- if eq .Role "user" }}<|start_header_id|>user<|end_header_id|>
{{- if and $.Tools $last }}

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.

{{ range $.Tools }}
{{- . }}
{{ end }}
Question: {{ .Content }}<|eot_id|>
{{- else }}

{{ .Content }}<|eot_id|>
{{- end }}{{ if $last }}<|start_header_id|>assistant<|end_header_id|>

{{ e

https://github.com/ollama/ollama/blob/main/docs/api.md#generate-a-completion

In [41]:
import ollama

# ollama keeps models 5 min in memory by default, they are reloaded by a query
def ollama_load(model):
    sampling_params = { "num_predict":1 }
    ollama.generate(model=model, prompt="load", raw=True, options=sampling_params, stream=False)
    return model

# ollama API only supports batch size 1
def ollama_generate(messages, model):
    print(f"ollama performance test:")
    
    prompts = format_prompt(messages, runtime._model)
    sampling_params = { "temperature":0.7, "top_p":0.8, "repeat_penalty":1.05, "num_predict":512 }
    # warmup
    output = ollama.generate(model=model, prompt=prompts[0], raw=True, options=sampling_params, stream=False)
    print(f"Generated text: {output['response']!r}")
    
    for msg_index in range(len(messages)):
        start_time = time.time()  # Record the start time
        output = ollama.generate(model=model, prompt=prompts[msg_index], raw=True, options=sampling_params, stream=False)
        end_time = time.time()  # Record the end time
            
        # Print the outputs.
        tokenscount = output['eval_count']
        tokens_per_sec = tokenscount/(end_time-start_time)
        print(f"- batch size 1: {tokens_per_sec:.2f} tokens/sec")

In [43]:
model = ollama_load(ollama_test_models["llama-3.1"])

In [44]:
ollama_generate(test_messages, model)

ollama performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre divers avantages à ses adhérents. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts réduits sur les prêts** : Le Crédit Mutuel propose des taux d'intérêt compétitifs pour les prêts personnels, hypothécaires et professionnels.\n2. **Avantages fiscaux** : Les adhérents du Crédit Mutuel peuvent bénéficier de réductions fiscales sur leurs prêts et épargnes, en fonction de leur situation financière et de leurs impôts.\n3. **Assistance financière** : Le Crédit Mutuel offre des services d'assistance financière pour aider ses adhérents à gérer leurs finances et à atteindre leurs objectifs économiques.\n4. **Produits diversifiés** : Le Crédit Mutuel propose une gamme complète de produits bancaires, tels que les comptes courants, les livrets d'épargne, les cartes de crédit, les assurances et les investissements.\n5. **Services en ligne** : Les adhérents du Crédit Mutuel ont accè

https://docs.vllm.ai/en/latest/quantization/fp8.html#quantization-process